In [78]:
"""
    separate the predictors aand the labels
"""
import pandas as pd
import numpy as np
import os

HOUSING_PATH = './'
def load_housing_data(housing_path = HOUSING_PATH):
    csv_path = os.path.join(housing_path,"housing.csv")
    return pd.read_csv(csv_path)

housing = load_housing_data()

"""cut the median_income into five categories"""
housing["income_cut"] = pd.cut(housing["median_income"],
                               bins=[0.,1.5,3.0,4.5,6.,np.inf],
                               labels=[1,2,3,4,5])

In [79]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1,test_size=0.2,random_state=42)
for train_index,test_index in split.split(housing,housing["income_cut"]):
    start_train_set = housing.loc[train_index]
    start_test_set = housing.loc[test_index]

In [80]:
"""
    prepare the data for algorithm
"""
housing = start_train_set.drop("median_house_value",axis=1)
housing_labels = start_train_set["median_house_value"].copy()

In [81]:
housing.describe()


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income
count,16512.000000,16512.000000,16512.000000,16512.000000,16354.000000,16512.000000,16512.000000,16512.000000
mean,-119.575834,35.639577,28.653101,2622.728319,534.973890,1419.790819,497.060380,3.875589
std,2.001860,2.138058,12.574726,2138.458419,412.699041,1115.686241,375.720845,1.904950
min,-124.350000,32.540000,1.000000,6.000000,2.000000,3.000000,2.000000,0.499900
25%,-121.800000,33.940000,18.000000,1443.000000,295.000000,784.000000,279.000000,2.566775
50%,-118.510000,34.260000,29.000000,2119.500000,433.000000,1164.000000,408.000000,3.540900
75%,-118.010000,37.720000,37.000000,3141.000000,644.000000,1719.250000,602.000000,4.744475
max,-114.310000,41.950000,52.000000,39320.000000,6210.000000,35682.000000,5358.000000,15.000100


In [82]:
"""
    #option 1 :housing.dropna(subset = ["total_bedrooms"])
    #option 2 :housing.drop("total_bedrooms",axis = 1)
"""
median = housing["total_bedrooms"].median()
housing["total_bedrooms"].fillna(median,inplace = True)
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity,income_cut
17606,-121.89,37.29,38.0,1568.0,351.0,710.0,339.0,2.7042,<1H OCEAN,2
18632,-121.93,37.05,14.0,679.0,108.0,306.0,113.0,6.4214,<1H OCEAN,5
14650,-117.20,32.77,31.0,1952.0,471.0,936.0,462.0,2.8621,NEAR OCEAN,2
3230,-119.61,36.31,25.0,1847.0,371.0,1460.0,353.0,1.8839,INLAND,2
3555,-118.59,34.23,17.0,6592.0,1525.0,4459.0,1463.0,3.0347,<1H OCEAN,3


In [83]:
"""
    How to use the sklearn to solve the missing data
"""
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="median")

housing_num = housing.drop("ocean_proximity",axis = 1)

imputer.fit(housing_num) # housing_num 's type is a dataframe
imputer.statistics_

array([-118.51  ,   34.26  ,   29.    , 2119.5   ,  433.    , 1164.    ,
        408.    ,    3.5409,    3.    ])

In [84]:
housing_num.median().values

array([-118.51  ,   34.26  ,   29.    , 2119.5   ,  433.    , 1164.    ,
        408.    ,    3.5409])

In [85]:
"""
    use imputer to transform the training set by replacing missing values
"""
X = imputer.transform(housing_num) # the result is numpy array

housing_tr = pd.DataFrame(X,columns=housing_num.columns,index = housing_num.index)

In [86]:
housing_cat = housing[["ocean_proximity"]]
housing_cat.head()

,ocean_proximity
17606,<1H OCEAN
18632,<1H OCEAN
14650,NEAR OCEAN
3230,INLAND
3555,<1H OCEAN


In [87]:
"""
    use sklearn's OrdinalEncode convert these categories from text
"""
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder()
housing_cat_encoded = ordinal_encoder.fit_transform(housing_cat)
housing_cat_encoded[:10]

array([[0.],
       [0.],
       [4.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.]])

In [88]:
ordinal_encoder.categories_

[array(['<1H OCEAN', 'INLAND', 'ISLAND', 'NEAR BAY', 'NEAR OCEAN'],
       dtype=object)]

In [89]:
"""
    OneHotEncoder class to convert categories values into one-hot values
"""
from sklearn.preprocessing import OneHotEncoder
cat_encoder = OneHotEncoder()
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
housing_cat_1hot

# output is a scipy sparse matrix instead of a numpy array

<16512x5 sparse matrix of type '<class 'numpy.float64'>'
	with 16512 stored elements in Compressed Sparse Row format>

In [90]:
"""
    convert it to a dense numpy array
"""
housing_cat_1hot.toarray()

array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       ...,
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.]])

In [91]:
cat_encoder.categories_

[array(['<1H OCEAN', 'INLAND', 'ISLAND', 'NEAR BAY', 'NEAR OCEAN'],
       dtype=object)]

In [92]:
from sklearn.base import BaseEstimator,TransformerMixin

rooms_ix,bedrooms_ix,population_ix,households_ix = 3,4,5,6
class CombinedAttributesAdder(BaseEstimator,TransformerMixin):
    # no *args or ** kargs
    def __init__(self,add_bedrooms_per_room = True):
        self.add_bedrooms_per_room = add_bedrooms_per_room

    def fit(self,X,y = None):
        return self # nothing else to do

    def transform(self,X,y = None):
        rooms_per_household = X[:,rooms_ix]/X[:,households_ix]
        population_per_household = X[:,population_ix]/X[:,households_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:,bedrooms_ix]/X[:,rooms_ix]
            return np.c_[X,rooms_per_household,population_per_household,bedrooms_per_room]
        else:
            return np.c_[X,rooms_per_household,population_per_household]

attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
housing_extra_attribs = attr_adder.transform(housing.values)
housing.values

array([[-121.89, 37.29, 38.0, ..., 2.7042, '<1H OCEAN', 2],
       [-121.93, 37.05, 14.0, ..., 6.4214, '<1H OCEAN', 5],
       [-117.2, 32.77, 31.0, ..., 2.8621, 'NEAR OCEAN', 2],
       ...,
       [-116.4, 34.09, 9.0, ..., 3.2723, 'INLAND', 3],
       [-118.01, 33.82, 31.0, ..., 4.0625, '<1H OCEAN', 3],
       [-122.45, 37.77, 52.0, ..., 3.575, 'NEAR BAY', 3]], dtype=object)

In [93]:
housing_extra_attribs

array([[-121.89, 37.29, 38.0, ..., 2, 4.625368731563422,
        2.094395280235988],
       [-121.93, 37.05, 14.0, ..., 5, 6.008849557522124,
        2.7079646017699117],
       [-117.2, 32.77, 31.0, ..., 2, 4.225108225108225,
        2.0259740259740258],
       ...,
       [-116.4, 34.09, 9.0, ..., 3, 6.34640522875817, 2.742483660130719],
       [-118.01, 33.82, 31.0, ..., 3, 5.50561797752809,
        3.808988764044944],
       [-122.45, 37.77, 52.0, ..., 3, 4.843505477308295,
        1.9859154929577465]], dtype=object)

In [94]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
    ('imputer',SimpleImputer(strategy="median")),
    ('attribs_adder',CombinedAttributesAdder()),
    ('std_scaler',StandardScaler()),
])
housing_num_tr = num_pipeline.fit_transform(housing_num)
housing_num_tr

array([[-1.15604281,  0.77194962,  0.74333089, ..., -0.31205452,
        -0.08649871,  0.15531753],
       [-1.17602483,  0.6596948 , -1.1653172 , ...,  0.21768338,
        -0.03353391, -0.83628902],
       [ 1.18684903, -1.34218285,  0.18664186, ..., -0.46531516,
        -0.09240499,  0.4222004 ],
       ...,
       [ 1.58648943, -0.72478134, -1.56295222, ...,  0.3469342 ,
        -0.03055414, -0.52177644],
       [ 0.78221312, -0.85106801,  0.18664186, ...,  0.02499488,
         0.06150916, -0.30340741],
       [-1.43579109,  0.99645926,  1.85670895, ..., -0.22852947,
        -0.09586294,  0.10180567]])

In [95]:
"""
    apply the appropriate transformers to each column
"""
from sklearn.compose import ColumnTransformer

num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

full_pipeline = ColumnTransformer([
    ("num",num_pipeline,num_attribs),
    ("cat",OneHotEncoder(),cat_attribs),
])

# 处理后的housing_prepared是ndarray
housing_prepared = full_pipeline.fit_transform(housing)
housing_prepared

array([[-1.15604281,  0.77194962,  0.74333089, ...,  0.        ,
         0.        ,  0.        ],
       [-1.17602483,  0.6596948 , -1.1653172 , ...,  0.        ,
         0.        ,  0.        ],
       [ 1.18684903, -1.34218285,  0.18664186, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 1.58648943, -0.72478134, -1.56295222, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.78221312, -0.85106801,  0.18664186, ...,  0.        ,
         0.        ,  0.        ],
       [-1.43579109,  0.99645926,  1.85670895, ...,  0.        ,
         1.        ,  0.        ]])

In [96]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(housing_prepared,housing_labels)

LinearRegression()

In [97]:
some_data = housing.iloc[:5]
some_labels = housing_labels.iloc[:5]

some_data_prepared = full_pipeline.transform(some_data)

print("Predictions:",lin_reg.predict(some_data_prepared))

Predictions: [203682.37379543 326371.39370781 204218.64588245  58685.4770482
 194213.06443039]


In [98]:
print("Labels:",list(some_labels))


Labels: [286600.0, 340600.0, 196900.0, 46300.0, 254500.0]


In [99]:
from sklearn.metrics import mean_squared_error
housing_predictions = lin_reg.predict(housing_prepared)
lin_mse = mean_squared_error(housing_labels,housing_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

68376.64295459937

In [100]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor()
tree_reg.fit(housing_prepared,housing_labels)

housing_predictions = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(housing_labels,housing_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

0.0

In [101]:
"""
    交叉验证 - K-fold cross-validation
"""
from sklearn.model_selection import cross_val_score
scores = cross_val_score(tree_reg,housing_prepared,
                         housing_labels,scoring="neg_mean_squared_error",cv = 10)
tree_rmse_scores = np.sqrt(-scores)

In [102]:
def display_scores(scores):
    print("Scores:",scores)
    print("Mean:",scores.mean())
    print("Standard deviation",scores.std())

display_scores(tree_rmse_scores)

Scores: [69715.51769022 64975.76635455 71263.11219403 69089.04238115
 71264.03553795 74657.38819783 72294.88907993 72497.90918128
 77257.77807504 69622.6369574 ]
Mean: 71263.80756493699
Standard deviation 3152.616639886209


In [103]:
"""
    检验线性回归模型 交叉验证的分数
"""
lin_scores = cross_val_score(lin_reg,housing_prepared,
                             housing_labels,scoring="neg_mean_squared_error",cv = 10)
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

Scores: [66877.52325028 66608.120256   70575.91118868 74179.94799352
 67683.32205678 71103.16843468 64782.65896552 67711.29940352
 71080.40484136 67687.6384546 ]
Mean: 68828.99948449331
Standard deviation 2662.761570610342


In [104]:
""" Random Forest"""
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor()
forest_reg.fit(housing_prepared,housing_labels)

housing_predictions = forest_reg.predict(housing_prepared)
forest_mse = mean_squared_error(housing_labels,housing_predictions)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

18660.993416405552

In [105]:
forest_scores = cross_val_score(forest_reg,housing_prepared,
                             housing_labels,scoring="neg_mean_squared_error",cv = 10)
forest_rmse_scores = np.sqrt(-forest_scores)
display_scores(forest_rmse_scores)

Scores: [49488.61309579 47607.22100081 49914.27847241 52553.49434429
 49660.3162428  53641.24362424 48398.76905347 47893.3959931
 53239.38743239 50446.25840771]
Mean: 50284.297766699456
Standard deviation 2068.208788429515


In [106]:
from sklearn.model_selection import GridSearchCV

param_grid  = [
    {'n_estimators':[3,10,30],'max_features':[2,4,6,8]},
    {'bootstrap':[False],'n_estimators':[3,10],'max_features':[2,3,4]}
]

forest_reg = RandomForestRegressor()
grid_search = GridSearchCV(forest_reg,param_grid,cv = 5,
                           scoring="neg_mean_squared_error",
                           return_train_score=True)
grid_search.fit(housing_prepared,housing_labels)

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid=[{'max_features': [2, 4, 6, 8],
                          'n_estimators': [3, 10, 30]},
                         {'bootstrap': [False], 'max_features': [2, 3, 4],
                          'n_estimators': [3, 10]}],
             return_train_score=True, scoring='neg_mean_squared_error')

In [107]:
grid_search.best_params_

{'max_features': 6, 'n_estimators': 30}

In [108]:
grid_search.best_estimator_

RandomForestRegressor(max_features=6, n_estimators=30)

In [109]:
"""
    get the evaluation scores
"""
cvres = grid_search.cv_results_
for mean_scores,params in zip(cvres["mean_test_score"],cvres["params"]):
    print(np.sqrt(-mean_scores),params)

63735.40365531066 {'max_features': 2, 'n_estimators': 3}
55752.9540402592 {'max_features': 2, 'n_estimators': 10}
53238.48880937571 {'max_features': 2, 'n_estimators': 30}
61814.010691432886 {'max_features': 4, 'n_estimators': 3}
53720.42677543225 {'max_features': 4, 'n_estimators': 10}
51465.875880889194 {'max_features': 4, 'n_estimators': 30}
60124.43292486328 {'max_features': 6, 'n_estimators': 3}
52974.114448533204 {'max_features': 6, 'n_estimators': 10}
50660.260402236185 {'max_features': 6, 'n_estimators': 30}
59567.993433312826 {'max_features': 8, 'n_estimators': 3}
52883.5744093446 {'max_features': 8, 'n_estimators': 10}
50987.72478043902 {'max_features': 8, 'n_estimators': 30}
62221.90846737412 {'bootstrap': False, 'max_features': 2, 'n_estimators': 3}
54847.40110602887 {'bootstrap': False, 'max_features': 2, 'n_estimators': 10}
60739.24599475075 {'bootstrap': False, 'max_features': 3, 'n_estimators': 3}
53548.94125551338 {'bootstrap': False, 'max_features': 3, 'n_estimators':

In [110]:
feature_importances = grid_search.best_estimator_.feature_importances_
feature_importances


array([6.63190979e-02, 6.16741699e-02, 4.30346457e-02, 1.72700880e-02,
       1.79965591e-02, 1.79405347e-02, 1.77115460e-02, 2.28320644e-01,
       1.88573816e-01, 4.16138746e-02, 1.03669492e-01, 4.51167756e-02,
       8.02635589e-03, 1.34485004e-01, 7.92720208e-05, 3.36847478e-03,
       4.79964936e-03])

In [111]:
extra_atrribs = ["rooms_per_hhold","pop_per_hhold","bedrooms_per_room"]
cat_encoder = full_pipeline.named_transformers_["cat"]
cat_one_hot_attribs = list(cat_encoder.categories_[0])
attributes = num_attribs + extra_atrribs + cat_one_hot_attribs

sorted(zip(feature_importances,attributes),reverse=True)

[(0.2283206437090644, 'median_income'),
 (0.18857381623988892, 'income_cut'),
 (0.13448500407019517, 'INLAND'),
 (0.10366949249162648, 'pop_per_hhold'),
 (0.06631909785413752, 'longitude'),
 (0.061674169863025534, 'latitude'),
 (0.04511677561688839, 'bedrooms_per_room'),
 (0.04303464569688774, 'housing_median_age'),
 (0.04161387462185507, 'rooms_per_hhold'),
 (0.01799655908541546, 'total_bedrooms'),
 (0.017940534678461177, 'population'),
 (0.01771154604619338, 'households'),
 (0.01727008796859366, 'total_rooms'),
 (0.008026355893269995, '<1H OCEAN'),
 (0.004799649363382247, 'NEAR OCEAN'),
 (0.003368474780303928, 'NEAR BAY'),
 (7.927202081089456e-05, 'ISLAND')]

In [114]:
final_model = grid_search.best_estimator_

X_test = start_test_set.drop("median_house_value",axis=1)
y_test = start_test_set["median_house_value"].copy()

X_test_prepared = full_pipeline.transform(X_test)
final_predictions = final_model.predict(X_test_prepared)

final_mse = mean_squared_error(y_test,final_predictions)
final_rmse = np.sqrt(final_mse) # evaluates to 47730.2
final_rmse

48754.278609914836

In [116]:
from scipy import stats
confidence = 0.95
squared_errors = (final_predictions - y_test)**2
np.sqrt(stats.t.interval(confidence,len(squared_errors)-1,loc = squared_errors.mean(),scale = stats.sem(squared_errors)))

array([46757.41347254, 50672.51376145])